In [2]:
from torchvision.models import mobilenet_v3_large
import torch
from torch.utils.data import DataLoader

torch.manual_seed(42)

images = torch.load("_03_baseline_cnn/data/images.pt")
labels = torch.load("_03_baseline_cnn/data/labels.pt")

dataset = torch.utils.data.TensorDataset(images, labels)

loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

In [3]:
model = mobilenet_v3_large(pretrained=True, weights='IMAGENET1K_V1')
print(model)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to C:\Users\Famiglia Christille/.cache\torch\hub\checkpoints\mobilenet_v3_large-8738ca79.pth


100%|██████████| 21.1M/21.1M [00:04<00:00, 4.50MB/s]


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [ ]:


for param in model.parameters():
    param.requires_grad = False

model.set_submodule('classifier', nn.Identity())

model.get_model_inputs_from_batch = (lambda batch: [batch[0]])
model.get_labels_from_batch = (lambda batch: batch[1])


with torch.no_grad():
    model.eval()
    for batch in loader:
        processed_images = model(*model.get_model_inputs_from_batch(batch))
        print(processed_images.shape)
        labels = model.get_labels_from_batch(batch)
        print(labels.shape)

        torch.save(processed_images, "03_baseline_cnn/data/extracted_features/basedata.pt")
        torch.save(labels, "03_baseline_cnn/data/extracted_features/basedata_labels.pt")

torch.Size([1000, 960])
torch.Size([1000])
